In [ ]:
#必要なモジュールをインポート
import RPi.GPIO as GPIO             #GPIO用のモジュールをインポート
import time                         #時間制御用のモジュールをインポート
import sys                          #sysモジュールをインポート
import requests　　　　　　　　　　 #repuestsモジュールのインポート（LINE）
GPIO.setwarnings(False)　　　　　　 #エラーが出てこないようにするため

#ポート番号の定義
Trig = 27                           #変数"Trig"に27を代入
Echo = 18                           #変数"Echo"に18を代入

#GPIOの設定
GPIO.setmode(GPIO.BCM)              #GPIOのモードを"GPIO.BCM"に設定
GPIO.setup(Trig, GPIO.OUT)          #GPIO27を出力モードに設定
GPIO.setup(Echo, GPIO.IN)           #GPIO18を入力モードに設定

def okaeri():　　　　　　　　　　　　　　　　#帰ってきたとき
    send_line_notify('帰ってきました')


def itterasyai():　　　　　　　　　　　　　　#出かけたとき
    send_line_notify('出かけました')


def send_line_notify(notification_message):
    """
    LINEに通知する
    """
    line_notify_token = 'トークン'
    line_notify_api = 'https://notify-api.line.me/api/notify'
    headers = {'Authorization': f'Bearer {line_notify_token}'}
    data = {'message': f'message: {notification_message}'}
    requests.post(line_notify_api, headers = headers, data = data)
    
#HC-SR04で距離を測定する関数
def read_distance():
    GPIO.output(Trig, GPIO.HIGH)            #GPIO27の出力をHigh(3.3V)にする
    time.sleep(0.00001)                     #10μ秒間待つ
    GPIO.output(Trig, GPIO.LOW)             #GPIO27の出力をLow(0V)にする
                   
    while GPIO.input(Echo) == GPIO.LOW:     #GPIO18がLowの時間
        sig_off = time.time()
    while GPIO.input(Echo) == GPIO.HIGH:    #GPIO18がHighの時間
        sig_on = time.time()

    duration = sig_on - sig_off             #GPIO18がHighしている時間を算術
    distance = duration * 34000 / 2         #距離を求める(cm)
    return distance
#連続して値を超音波センサの状態を読み取る
while True:
    try:
        cm = read_distance()                   #HC-SR04で距離を測定する
        print("distance=", int(cm), "cm")  #距離をint型で表示
        if cm > 2 and cm < 15:                #距離が2～15cmの場合
            okaeri()                       #okaeriの実行
            time.sleep(60)                        #1分間待つ
        else:
            itterasyai()　　　　　　　　　#itterasyaiの実行
            time.sleep(60)
            
except KeyboardInterrupt:       #Ctrl+Cキーが押された
        GPIO.cleanup()              #GPIOをクリーンアップ
        sys.exit()                  #プログラム終了